# Downloading QUANDL package

### As data is taken from quandl package

In [1]:
pip install quandl

SyntaxError: invalid syntax (<ipython-input-1-919755a5b3ab>, line 1)

# imports needed for project

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import date, datetime
import datetime as dt
import numpy as np
import quandl

# Loading downloading data from quandl and loading it into _`data`_ dataFrame


In [ ]:
data = quandl.get("WIKI/GOOGL", start_date="2004-08-19", end_date="2017-11-04")

# Displaying the dataFrame



In [2]:
data

NameError: name 'data' is not defined

In [ ]:
data.index

# Creating 2 columns derived from other columns

$$HL-PCT = \frac{(Adj. High - Adj. Low) * 100}{Adj.Close}$$

$$PCT-change = \frac{(Adj. Close - Adj. Open) * 100}{Adj.Open}$$



In [ ]:
data['HL_PCT'] = (data['Adj. High'] - data['Adj. Low']) * 100 / data['Adj. Close']
data['PCT_change'] = (data['Adj. Close'] - data['Adj. Open']) * 100 / data['Adj. Open']

In [ ]:
data

### Droping the rows with value as _`NA`_

In [ ]:
data.dropna(inplace=True)
data.info()

In [ ]:
data.describe()

In [ ]:
df = pd.DataFrame(data, columns=['Close'])

In [ ]:
df['Date'] = df.index

In [ ]:
for i in range(len(df['Date'])):
  df['Date'].iloc[i] = str(df['Date'].iloc[i])[:10]

In [ ]:
df.isna().values.any()

In [ ]:
df.set_index([pd.Index(np.arange(0,3328))], inplace=True)

# Plotting the Graph of Date vs Closing value of stock

In [ ]:
import matplotlib.dates as mdates

years = mdates.YearLocator() # Get every year
yearsFmt = mdates.DateFormatter('%Y') # Set year format

# Create subplots to plot graph and control axes
fig, ax = plt.subplots(figsize=(24,12))

ax.plot(df['Date'], df['Close'])

# Format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)

# plt.figure(figsize=(12, 12))
# Set figure title
plt.title('Close Stock Price History [2004 - 2017]', fontsize=18)
# Set x label
plt.xlabel('Date', fontsize=14)
# Set y label
plt.ylabel('Closing Stock Price ', fontsize=18)

# Rotate and align the x labels
fig.autofmt_xdate()

# Show plot
plt.show()

# Plotting heatmap for correlation between all the columns in the table

In [ ]:
data_copy=data.copy()
data_copy.drop(['Split Ratio'],axis=1,inplace=True)
plt.subplots(figsize=(12,12))
sns.heatmap(data_copy.corr())


In [ ]:
displayedClose = list()
count = 0
for i in data_copy['Close']:
  if count == 125 or count == 0:
    displayedClose.append(i)
    count = 1
  count += 1
print(displayedClose)

# Normalizing all the columns

In [ ]:
for column in data.columns:
    data[column] = data[column]  / data[column].abs().max()

In [ ]:
data

# Splitting data for training and testing

In [ ]:
x = data[['Adj. Close',	'Adj. Volume',	'HL_PCT',	'PCT_change']]
y = data['Close']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.3)

In [ ]:
x_test.sort_index(inplace=True)
y_test.sort_index(inplace=True)

# Linear Regression Scores on training and testing data

In [ ]:
Lreg = LinearRegression().fit(x_train, y_train)
print(Lreg.score(x_train, y_train))
print(Lreg.score(x_test, y_test))

In [ ]:
y_pred = Lreg.predict(x_test)
import math
from sklearn.metrics import mean_squared_error ,r2_score
print("mean squared error MSE %.2f" %mean_squared_error(y_test,y_pred))
print("root mean RMSE %.2f" % math.sqrt(mean_squared_error(y_test,y_pred)))
print("mean RMSE %.5f" % r2_score(y_test,y_pred))

In [ ]:
displayedDate = list()
count = 0
for i in data.index:
  if count == 125 or count == 0:
    i = datetime.date(i)
    displayedDate.append(i)
    count = 1
  count += 1
print(displayedDate)

# Plotting the Linear Regression line on the stock price graph using _sns regplot_

In [ ]:
plt.figure(figsize=(24, 12))
# plt.scatter(data.index, data['Close'], s=1, color='red')
# plt.plot(x_test.index, y_pred, color='blue')
sns.regplot(data.index.map(dt.datetime.toordinal), data['Close'] ,marker=".")

# Plotting the Linear Regression line on the stock price graph using _custom linear regression_

In [ ]:
train, test = train_test_split(df, test_size=0.20)

In [ ]:
X_train_self = np.array(train.index).reshape(-1, 1)
y_train_self = train['Close']

In [ ]:
model = LinearRegression()
# Fit linear model using the train data set
model.fit(X_train_self, y_train_self)

In [ ]:
plt.figure(1, figsize=(24,12))
plt.title('Linear Regression | Price vs Time')
plt.scatter(X_train_self, y_train_self, edgecolor='w', label='Actual Price')
plt.plot(X_train_self, model.predict(X_train_self), color='r', label='Predicted Price')
plt.xlabel('Integer Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(24, 12))
plt.plot(data.index, data['Close'], color='red')
plt.scatter(x_test.index, y_pred, s=2,  color='blue')
# plt.plot(testing.index, y_pred_test, color='green')

# Loading unseen testing data

In [ ]:
testing = quandl.get("WIKI/GOOGL", start_date="2017-10-28", end_date="2018-03-27")

In [ ]:
testing

In [ ]:
testing['HL_PCT'] = (testing['Adj. High'] - testing['Adj. Low']) * 100 / testing['Adj. Close']
testing['PCT_change'] = (testing['Adj. Close'] - testing['Adj. Open']) * 100 / testing['Adj. Open']
for column in testing.columns:
    testing[column] = testing[column]  / testing[column].abs().max()
y_pred_test = Lreg.predict(testing[['Adj. Close', 'HL_PCT', 'PCT_change','Adj. Volume']])
y_test_test = testing['Close']
# print(y_test_test, y_pred)
import math
from sklearn.metrics import mean_squared_error ,r2_score
print("mean squared error MSE %.2f" %mean_squared_error(y_test_test,y_pred_test))
print("root mean RMSE %.2f" % math.sqrt(mean_squared_error(y_test_test,y_pred_test)))
print("mean RMSE %.5f" % r2_score(y_test_test,y_pred_test))


# Plotting future prediction value of stocks

In [ ]:
# for column in data.columns:
#     data[column] = data[column]  * 100 / data[column].abs().min()

# for column in testing.columns:
#     testing[column] = testing[column] * 100 / testing[column].abs().min()
plt.figure(figsize=(24, 12))
plt.plot(data.index, data['Close'], color='red')
plt.scatter(x_test.index, y_pred, s=2,  color='blue')
plt.scatter(testing.index, y_pred_test, s=2, color='green')
# plt.plot(testing.index, y_test_test)

# Comparing some predicted values and actual values using bar plot

In [ ]:
X_test_self = np.array(test.index).reshape(-1, 1)
y_test_self = test['Close']
model.score(X_test_self, y_test_self)

In [ ]:
y_pred_train = model.predict(X_test_self)

In [ ]:
y_pred_train.shape

In [ ]:
randints = np.random.randint(2550, size=25)

# Select row numbers == random numbers
df_sample = df[df.index.isin(randints)]
df_sample['Prediction'] = y_pred_train[:25]

In [ ]:
# Create subplots to plot graph and control axes
fig, ax = plt.subplots(figsize=(24, 12))
df_sample.plot(x='Date', y=['Close', 'Prediction'], kind='bar', ax=ax)

# Set figure title
plt.title('Comparison Predicted vs Actual Price in Sample data selection', fontsize=16)

# 

# Set x label
plt.xlabel('Date', fontsize=14)

# Set y label
plt.ylabel('Stock Price in $', fontsize=14)

# Show plot
plt.show()

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 3)
knn.fit(x_train,y_train)

# knn.predict(x_test).shape

In [ ]:
knn.score(x_train,y_train)

In [ ]:
knn.score(x_test, y_test)

In [ ]:
y_pred_knn = knn.predict(testing[['Adj. Close', 'HL_PCT', 'PCT_change','Adj. Volume']])

In [ ]:
plt.figure(figsize=(24, 12))
plt.plot(data.index, data['Close'], color='red')
# plt.scatter(x_test.index, y_pred, s=2,  color='blue')
plt.scatter(x_test.index, knn.predict(x_test), s=2, color='blue')
plt.scatter(testing.index, y_pred_knn, s=2, color='green')
# plt.plot(testing.index, y_test_test)

# Desicion Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
best_score = 0

for depth in range(2,10):
    for split in range(2,5):
        for leaf in range(5,20):
            tree_grid_search = DecisionTreeRegressor(
                                                      max_depth = depth,
                                                      min_samples_split = split,
                                                      max_leaf_nodes= leaf)
            tree_grid_search.fit(x_train,y_train)
            
            
            test_score = tree_grid_search.score(x_test,y_test)
            train_score = tree_grid_search.score(x_train,y_train)
#                 parameters = {'Depth':depth,
#                               'Split':split,
#                               'Leaves':leaf,
#                               'Criterion':criteria}
#                 print("Parameters = ",parameters)
            if test_score > best_score:
                best_score = test_score
                best_parameters = {'Depth':depth,
                                  'Split':split,
                                  'Leaves':leaf,
                                  'Criterion':criteria}
print("Best Score: "+str(best_score))
print("Best Parameters: "+str(best_parameters))

In [ ]:
from sklearn.model_selection import GridSearchCV
depth = list(range(2,10))
split = list(range(2,5))
leaf = list(range(5,20))

param_grid = {'max_depth':depth,
             'min_samples_split':split,
             'max_leaf_nodes':leaf}

tree_grid_cv = DecisionTreeRegressor()

grid_search = GridSearchCV(tree_grid_cv , param_grid,cv=5)

grid_search.fit(x_train,y_train)

In [ ]:
print("Test acc: "+str(grid_search.score(x_test,y_test)))

In [ ]:
print("Best parameter :" +str(grid_search.best_params_))
print("Best score: "+str(grid_search.best_score_))